In [1]:
training='train.txt'
#training set tokens
tokens=[]

with open(training,'r',encoding='utf-8') as t:
  for line in t:
    each_line_tokens=line.lower().strip().split()
    tokens_per_line=['<s>']+ each_line_tokens+['</s>']
    tokens.append(tokens_per_line)

In [2]:
validation='val.txt'
# test set tokens
validation_tokens=[]

with open(validation,'r',encoding='utf-8') as v:
  for line in v:
    each_line_tokens=line.lower().strip().split()
    tokens_per_line=['<s>']+ each_line_tokens+['</s>']
    validation_tokens.append(tokens_per_line)


In [3]:
unigram_counts={}
total_word_count=0
# unigram model
for tokens_per_line in tokens:
  for token in tokens_per_line:
    if token not in unigram_counts:
      unigram_counts[token]=1
    else:
      unigram_counts[token]+=1
    total_word_count+=1

unigram_model={}
for word, count in unigram_counts.items():
  unigram_model[word]=count/total_word_count

unsmoothened_unigram_model=unigram_model.copy()


In [4]:
bigram_counts={}
#bigram_model
for tokens_per_line in tokens:
  for i in range(len(tokens_per_line)-1):
    bigram=(tokens_per_line[i],tokens_per_line[i+1])
    if bigram not in bigram_counts:
      bigram_counts[bigram]=1
    else:
      bigram_counts[bigram]+=1


bigram_model={}
for bigram, count in bigram_counts.items():
  first_word=bigram[0]
  bigram_model[bigram]=count/unigram_counts[first_word]

unsmoothened_bigram_model=bigram_model.copy()


In [6]:
# first method to handle unkown words
common_words={}

for word, count in unigram_counts.items():
  if count>1:
    common_words[word]=count

tokens_with_unk=tokens.copy()

for tokens_per_line in tokens_with_unk:
  for i in range(len(tokens_per_line)):
    if tokens_per_line[i] not in common_words:
      tokens_per_line[i]='<unk>'

In [ ]:
unigram_counts_with_unk={}
total_word_count_with_unk=0
for tokens_per_line in tokens_with_unk:
  for token in tokens_per_line:
    if token not in unigram_counts_with_unk:
      unigram_counts_with_unk[token]=1
    else:
      unigram_counts_with_unk[token]+=1
    total_word_count_with_unk+=1

unigram_model={}
for unigram, count in unigram_counts_with_unk.items():
  unigram_model[unigram]=count/total_word_count_with_unk

unigram_model_with_unk=unigram_model.copy()

In [ ]:
bigram_counts_with_unk={}
for tokens_per_line in tokens_with_unk:
  for i in range(len(tokens_per_line)-1):
    bigram=(tokens_per_line[i],tokens_per_line[i+1])
    if bigram not in bigram_counts_with_unk:
      bigram_counts_with_unk[bigram]=1
    else:
      bigram_counts_with_unk[bigram]+=1

bigram_model={}
for bigram, count in bigram_counts_with_unk.items():
  first_word=bigram[0]
  bigram_model[bigram]=count/unigram_counts_with_unk[first_word]

bigram_model_with_unk=bigram_model.copy()

In [ ]:
#second unknown words handling subword tokenization

In [ ]:
#Laplace smoothing
unigram_model={}
for word, count in unigram_counts_with_unk.items():
  unigram_model[word]=(count+1)/(total_word_count_with_unk+len(unigram_counts_with_unk))
  unigram_model_laplace=unigram_model.copy()

bigram_model={}
for bigram, count in bigram_counts_with_unk.items():
  first_word=bigram[0]
  bigram_model[bigram]=(count+1)/(unigram_counts_with_unk[first_word]+len(unigram_counts_with_unk))
  bigram_model_laplace=bigram_model.copy()

In [ ]:
#K-smoothing

unigram_model={}
k=0
for word, count in unigram_counts_with_unk.items():
  unigram_model[word]=(count+k)/(total_word_count_with_unk+k*len(unigram_counts_with_unk))
  unigram_model_laplace_k=unigram_model.copy()

bigram_model={}
for bigram, count in bigram_counts_with_unk.items():
  first_word=bigram[0]
  bigram_model[bigram]=(count+k)/(unigram_counts_with_unk[first_word]+k*len(unigram_counts_with_unk))
  bigram_model_laplace_k=bigram_model.copy()

In [ ]:
#perplexity_unigram_model

import math

def perplexity_calculator(validation_data, model,vocabulary):
  log_prob_sum=0.0
  N=0
  for review_tokens in validation_data:
    for token in review_tokens:
      if token=='<s>':
        continue

      N+=1

      lookup_token=token
      if lookup_token not in vocabulary:
        lookup_token='<unk>'

      probablity=model.get(lookup_token,0)

      if probablity==0:
        print(f"Error: Unknown word '{token}' found. Perplexity is infinite.")
        return float('inf')

      log_prob_sum+=math.log(probablity)

  avg_log_prob=log_prob_sum/N
  perplexity=math.exp(-avg_log_prob)
  return perplexity


In [ ]:
print(perplexity_calculator(validation_tokens,unigram_model_laplace_k,unigram_counts_with_unk))

293.2282067617357


In [ ]:
#perplexity_bigram_model

import math

def bigram_perplexity_calculator(validation_data, bigram_model, vocabulary):
  V=len(set(vocabulary.keys()))
  log_prob_sum=0.0
  N=0

  for review in validation_data:
    review = [word if word in vocabulary else '<unk>' for word in review]

    for i in range(len(review)-1):
      bigram=(review[i],review[i+1])

      if(bigram in bigram_model):
        probablity=bigram_model[bigram]
      else:
        probablity=1/(vocabulary.get(review[i],0)+V)


      N+=1
      log_prob_sum+=math.log(probablity)
    N+=len(review)-1

  if N==0:
    return float('inf')

  perplexity = math.exp(-log_prob_sum / N)
  return perplexity




In [ ]:
print(bigram_perplexity_calculator(validation_tokens,bigram_model_laplace,unigram_counts_with_unk))

20.714344358932546
